In [1]:
import gzip
import pandas as pd

In [3]:
with gzip.open('query_popularity.csv.gz') as f:
    query_popularity = pd.read_csv(f, sep='\t', escapechar='\\')

In [4]:
query_popularity.shape

(295675, 2)

In [5]:
query_popularity.head()

,query,query_popularity
0,ноутбук,10
1,куртка женская осенняя,10
2,ботинки женские,10
3,видеокарта,10
4,пальто женское осеннее,10


In [6]:
query = query_popularity['query'].values
query[1000:2000]

array(['майка мужская', 'сандалии мужские летние', 'шуба натуральная',
       'кольцо золотое с бриллиантом', 'пуховик женский турция',
       'елка новогодняя искусственная', 'пальто женское классическое',
       'брюки женские утепленные', 'цифровое фортепиано',
       'зимние ботинки для женщин', 'жилетка женская болоньевая',
       'поп ит', 'сарафан женский офисный', 'обогреватель',
       'тени для век', 'для собак', 'полупальто женское демисезонное',
       'garrett ace 400i', 'ботинки мужские натуральная кожа',
       'шляпа женская осенняя', 'водолазка женская теплая',
       'тайтсы женские', 'пальто женские зимние стеганые',
       'женская ортопедическая обувь зимняя', 'спортивные штаны мужские',
       'кроссовки женские обувь', 'перчатки женские натуральная кожа',
       'сковорода', 'телевизор lg 32', 'трусы женские набор',
       'пиджак женский удлиненный', 'менажница', 'плед 150х200',
       'худи женское теплое', 'пижама женская теплая', 'АЙФОН',
       'Philips 32PH

In [8]:
with gzip.open('search_history.csv.gz') as f:
    search_history = pd.read_csv(f, sep='\t', escapechar='\\')

In [9]:
search_history.shape

(73067895, 6)

In [10]:
search_history.head()

,wbuser_id,UQ,cnt,locale,weekday,time
0,37bc0ce12ffabce1b1882e66d461ed0e,тапочки женские домашние,1933,Ru,0,10:48:53
1,4636a6706e6736d818816d8657565aa2,чехол для бейджика,1513,Ru,0,10:48:53
2,708f4040baf99acfc9496563edff1b1a,GUESS,4,Ru,0,10:48:53
3,70311ec9008a31f743c164e6f1198c86,штора для ванной,0,Ru,0,10:48:53
4,70311ec9008a31f743c164e6f1198c86,фототфон,92272,Ru,0,10:48:53


In [11]:
uq = search_history['UQ'].values
uq[0:500]

array(['тапочки женские домашние', 'чехол для бейджика', 'GUESS',
       'штора для ванной', 'фототфон', '7024', 'туфли мужские осенние',
       'леггинсы женские', 'чехол на айфон 11',
       'ботинки женские мембрана', 'обувь женская ОКСФОРДЫ',
       'противоударный чехол на айфон 6',
       'зимняя обувь для мальчиков 37-38 размеров', 'бисер', 'робот змея',
       'костюм на флисе женский', 'светящийся ошейник для собак',
       'изделия из мед сплава', 'ремкомплект раздатки Нива Chevrolet',
       'оружие на лёго человека', 'костюмы женские классические',
       'масло моторное', 'свечи восковые', 'Бигуди на липучках',
       'шарф детский', 'сумка женская кожаная', 'испанский язык попова',
       'Starting6 Сборник', 'смартфон', 'шапка для девочек', 'Сяоми',
       'махровое олетенце 70х140', 'Набор для суши на 2 персоны',
       'колготки женские', 'постельное белье 2 спальное',
       'ариэль порошок', 'ангелочек статуэтка', 'памперсы 17 кг', 'esis',
       'калькулятор настоль

# faiss

In [12]:
from abc import ABCMeta, abstractmethod

from copy import deepcopy
import json


import numpy as np

import re
import os

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import collections

from pprint import pprint

import pickle


from tqdm.auto import tqdm
import faiss
import gc

from sentence_transformers import SentenceTransformer

sentence_tr_path = 'paraphrase-multilingual-MiniLM-L12-v2'

In [13]:
class FAISS:
    def __init__(self, dimensions:int):
        self.dimensions = dimensions
        self.index = faiss.IndexFlatL2(dimensions)
        self.vectors = {}
        self.counter = 0
        # self.model_name = 'distilbert-multilingual-nli-stsb-quora-ranking'
        self.model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
        self.sentence_encoder = SentenceTransformer(self.model_name)
    
    def add(self, text, idx):
        text_vec = self.sentence_encoder.encode([text])
        # print(type(text_vec))
        try:
            self.index.add(text_vec)
            self.vectors[self.counter] = (idx, text, text_vec)
            self.counter += 1
        except:
            print(f'text: {text}')
            print(f'text_vec: {text_vec}')
            
        
    def search(self, v:list, k:int=10):
        res = []
        distance, item_index = self.index.search(v, k)
        for dist, i in zip(distance[0], item_index[0]):
            if i==-1:
                break
            else:
                res.append((self.vectors[i][0], self.vectors[i][1], dist))
                    
        return res

In [14]:
def search(s, indexs, s_enc, k=10): 
    emb = s_enc.encode([s])
    r = indexs.search(emb, k)
    return r

In [15]:
index = FAISS(384)
for idx, q in tqdm(enumerate(query), total=len(query)):
    if type(q) == str:
        index.add(q, idx)        

Downloading: 100%|██████████| 968/968 [00:00<00:00, 967kB/s]
Downloading: 100%|██████████| 3.79k/3.79k [00:00<00:00, 1.89MB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 115kB/s]
Downloading: 100%|██████████| 471M/471M [00:34<00:00, 13.8MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 26.5kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:00<00:00, 13.4MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 119kB/s]
Downloading: 100%|██████████| 9.08M/9.08M [00:00<00:00, 11.3MB/s]
Downloading: 100%|██████████| 480/480 [00:00<00:00, 237kB/s]
Downloading: 100%|██████████| 14.8M/14.8M [00:00<00:00, 17.4MB/s]
 62%|██████▏   | 183235/295675 [1:16:14<46:47, 40.05it/s]  


KeyboardInterrupt: 

In [16]:
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
sentence_encoder = SentenceTransformer(model_name)

In [3]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doc
segmenter = Segmenter()
morphology_vocab = MorphVocab()
embeddings = NewsEmbedding()
morphology_tagger = NewsMorphTagger(embeddings)

def normalize(text: str) -> str:
    natasha_text = Doc(text)
    natasha_text.segment(segmenter)

    natasha_text.tag_morph(morphology_tagger)

    result = []
    for token in natasha_text.tokens:
        if token.pos != 'VERB':
            result.append(token.text)

    return " ".join(result)

print(normalize("купить семена"))
print(normalize("поясная сумочка"))


семена
поясная сумочка


In [17]:
n = np.random.randint(0, len(query))
# test = query[n]
test = 'поясная сумка'
print(test)
pprint(search(test, index, sentence_encoder), width=150)

поясная сумка
[(6848, 'поясная сумка', 0.0),
 (19478, 'сумка поясная', 0.2797336),
 (138019, 'поясная сумочка', 0.6349414),
 (109875, 'Вязаная сумка', 0.72920537),
 (77553, 'поясная сумка nike', 1.1642523),
 (11682, 'сумка на пояс', 1.348146),
 (54793, 'стеганая сумка', 1.5027943),
 (52216, 'стеганная сумка', 1.5389923),
 (154103, 'сумочка на пояс', 1.5410484),
 (135230, 'поясная сумка цепью', 1.5634053)]


In [ ]:
import natasha


n = np.random.randint(0, len(uq))
# test = uq[n]
test = 'купить семена'
print(test)
pprint(search(test, index, sentence_encoder, 50), width=150)

купить семена
[(336678, 'щавель семена', 4.545506),
 (326256, 'гацания семена', 4.6398964),
 (108979, 'укроп семена', 4.650968),
 (230169, 'семена чиа питание', 4.824304),
 (335099, 'зелнь семена', 4.8439736),
 (187731, 'укропа семена', 4.95915),
 (291514, 'помидор семена', 4.9630637),
 (93104, 'фацелия семена', 4.971968),
 (336842, 'Буденовка семена', 4.977025),
 (334072, 'люпин семена', 5.056629),
 (320979, 'черри семена', 5.064037),
 (160427, 'расторопша семена', 5.0680637),
 (88165, 'зира семена', 5.1489773),
 (327133, 'просо семена для проращивания', 5.1495275),
 (333057, 'петрушка семена', 5.169128),
 (147635, 'мята семена', 5.1798525),
 (335413, 'мак семена для посадки', 5.2006445),
 (324032, 'мелисса семена', 5.215031),
 (82241, 'семена горчицы', 5.2335987),
 (316850, 'семена чиа для проращивания', 5.2474375),
 (70894, 'кунжут семена', 5.2538633),
 (78448, 'для семян проращивание', 5.259571),
 (270684, 'семена мята', 5.295811),
 (237640, 'проращиватель семян', 5.3026824),
 (180